# __Merging Price & Patent Data__

* Cleaned_Price_Data - containing the drug name comparison information, and price information
* Cleaned_Patent_Data - containing product, patent, and exclusivity data

This file will merge these two datasets, and further clean/prep them for analysis. 

In [2]:
import pandas as pd
import fuzzywuzzy as fuzzy #We'll use this library to measure Levenshtein distance of drug names for matching
import datetime as dt

In [ ]:
# Reading in the data
Cleaned_Price_Data = pd.read_csv('data/Cleaned_Price_Data.csv')

In [4]:
Cleaned_Patent_Data = pd.read_csv('data/Cleaned_Patent_Data.csv')

In [5]:
# Drop old index columns
def drop_unnamed(df):
    for col in df.columns:
        if ('unnamed' in col) or ('Unnamed' in col):
            df.drop(col, axis = 1, inplace = True)
            
drop_unnamed(Cleaned_Price_Data)           
drop_unnamed(Cleaned_Patent_Data)           

In [7]:
# Set Price date-types
Cleaned_Price_Data['effective_date'] = pd.to_datetime(Cleaned_Price_Data['effective_date'], errors = 'coerce')
Cleaned_Price_Data['as_of_date'] = pd.to_datetime(Cleaned_Price_Data['as_of_date'], errors = 'coerce')

In [8]:
# And Patent date-types
Cleaned_Patent_Data['approval_date'] = pd.to_datetime(Cleaned_Patent_Data['approval_date'], errors = 'coerce')

In [9]:
# Sort dates
Cleaned_Price_Data = Cleaned_Price_Data.sort_values(by = ['ndc'])

### __Apply the row-adding function to the whole dataset__
Medicaid has stated that price data is measured on (roughly) a weekly basis.  The FDA's patent data does not follow that same schedule.  

Consequently, in order to merge the datasets by not only name but also by date, we'll need to add an observation (row) for each day so that I can properly match up drugs and dates in the prices dataset with those in the patent dataset.  The dates we're interested are: 

   * `effective_date` - the day the price is given
   * `approval_date` - the day the patent is approved

In [10]:
# Adds rows for all missing (daily) dates to each drug (by ndc)
def expand_dates(ser):
    return pd.DataFrame({'effective_date': pd.date_range(ser['effective_date'].min(), ser['effective_date'].max(), freq='D')})

price_cols = list(Cleaned_Price_Data.columns)

all_effective_dates = Cleaned_Price_Data.groupby(['ndc']).apply(expand_dates).reset_index().merge(Cleaned_Price_Data, how = 'left')[price_cols].ffill()
all_effective_dates.drop_duplicates(inplace = True)

len(all_effective_dates)

55966535

In [12]:
# Pickle data
import dill
dill.dump(all_effective_dates, open('temp_all_effective_dates.pkd', 'wb'))

In [33]:
# Unpickle data
import dill
all_effective_dates = dill.load(open('data/temp_all_effective_dates.pkd', 'rb'))
# del Cleaned_Price_Data # replaced by all_effective_dates

In [13]:
all_effective_dates.effective_date.value_counts(ascending=False)

2018-10-17    128805
2018-01-17    127044
2016-10-19    123129
2018-12-19    118499
2017-07-19    118433
               ...  
2013-02-27       331
2013-02-28       331
2013-02-25       329
2013-02-23       329
2013-02-24       329
Name: effective_date, Length: 2401, dtype: int64

### __Drop duplicates and as_of_date column in price data__

In [34]:
# as_of_date column is unneeded and solicited duplicate information (i.e. observations/rows for which we already have a date for each drug)
all_effective_dates.drop('as_of_date', inplace = True, axis = 1)
all_effective_dates.drop_duplicates(inplace = True)
len(all_effective_dates)

50241500

### __Start fuzzy matching process__
Create unique lists of ndc_description and ndc_description_agg.  Use fuzzy matching on those two lists to create a dictionary, then merge on the dictionary

In [ ]:
#Get a list of unique values in each column
ndc_description = list(all_effective_dates['ndc_description'].unique())
ndc_description_agg = list(Cleaned_Patent_Data['ndc_description_agg'].unique())

In [ ]:
print(' Unique drugs in ndc_description: ', len(ndc_description),
      '\n', 'Unique drugs in ndc_description_agg:', len(ndc_description_agg))

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import defaultdict

name_matches = {}
for price_name in ndc_description:
    for patent_name in ndc_description_agg:
        score = fuzz.token_sort_ratio(price_name, patent_name)
        if score > 85:
            name_matches[price_name] = patent_name
        else:
            continue
len(name_matches) #Format: {price_names : patent_names}

### __Un/pickle Data Here__

In [ ]:
# Pickle name-matching dictionary (took about 18 hours to create matches) - with the length being exactly 500, I'm skeptical it's complete...
import dill
dill.dump(name_matches, open('data/drug_name_matches.pkd', 'wb'))

In [11]:
# Unpickle name-matching dictionary
import dill
name_matches = dill.load(open('data/drug_name_matches.pkd', 'rb'))
# name_matches

### __Map the name_matches dictionary to price dataframe__

This preps the price data so that it has a common column to merge with the patent data.

Also, doing a bit more preliminary work before merging...

In [35]:
# Sort by ndc (and thus patent_names), and effective_date
all_effective_dates = all_effective_dates.sort_values(by = ['ndc', 'effective_date'])

In [36]:
#Match up names in Price column with names in dictionary derived from fuzzy name matching
all_effective_dates['patent_names'] = all_effective_dates['ndc_description'].map(name_matches)

In [37]:
# Convert all blanks to Nulls
all_effective_dates[all_effective_dates['patent_names'] == ''] = None

In [38]:
# Count nulls in patent_name column
all_effective_dates['patent_names'].isnull().value_counts(dropna = False)

True     43495182
False     6746318
Name: patent_names, dtype: int64

In [42]:
all_effective_dates.update(all_effective_dates.groupby('ndc')['patent_names'].fillna(method = 'bfill', axis = 0, inplace = True)) #forward fill names --> apparently didn't do anything...

In [44]:
# Pickle data
import dill
dill.dump(all_effective_dates, open('temp_all_effective_dates.pkd', 'wb'))

In [45]:
# Drop rows without patent_names value (basically rows with dates only)
import numpy as np
all_effective_dates.dropna(subset = ['patent_names'], inplace = True, axis = 0)
all_effective_dates['patent_names'].isnull().value_counts(dropna = False)

False    6746318
Name: patent_names, dtype: int64

### __Set indexes to improve merging__

In [46]:
# Price data index
all_effective_dates.set_index('patent_names', inplace = True)

# Patent data index
Cleaned_Patent_Data.set_index('ndc_description_agg', inplace = True)
Cleaned_Patent_Data.index.names = ['patent_names'] #set the name of the index

## __Merge Datasets__

It's important that the merge occur on two columns (the drug name and date columns of each dataset).  These two columns form a unique 'key' for the merge, without which, Pandas may not merge the patent information to the correct `effective_date`.

We'll merge left because we want to keep all price data and just add in patent data where applicable as patent information (an independent variable) is of no value without price data (our dependent variable). 

In [47]:
# Merging left now because we want to keep all price data and just add in patent data where applicable.  Should have the same number of columns at end.
Price_Patent_Data = pd.merge(all_effective_dates, Cleaned_Patent_Data, how = 'left', left_on = [all_effective_dates.index, 'effective_date'], right_on = [Cleaned_Patent_Data.index, 'approval_date'])
Price_Patent_Data.head()

,key_0,classification_for_rate_setting,corresponding_generic_drug_effective_date,corresponding_generic_drug_nadac_per_unit,effective_date,explanation_code,nadac_per_unit,ndc,ndc_description,otc,...,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date,dosage_form,route
0,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-21,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-22,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-23,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-24,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-25,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
Price_Patent_Data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6746732 entries, 0 to 6746731
Data columns (total 39 columns):
key_0                                        6746732 non-null object
classification_for_rate_setting              6746732 non-null object
corresponding_generic_drug_effective_date    6746732 non-null object
corresponding_generic_drug_nadac_per_unit    6746732 non-null float64
effective_date                               6746732 non-null datetime64[ns]
explanation_code                             6746732 non-null object
nadac_per_unit                               6746732 non-null float64
ndc                                          6746732 non-null float64
ndc_description                              6746732 non-null object
otc                                          6746732 non-null object
pharmacy_type_indicator                      6746732 non-null object
pricing_unit                                 6746732 non-null object
ingredient                                   125

### __Export/Pickle the results__

In [ ]:
# Export to csv
Price_Patent_Data.to_csv('data/Price_Patent_Data.csv', index = False)

In [51]:
# Pickle data (quicker than writing to CSV)
dill.dump(Price_Patent_Data, open('data/Price_Patent_Data.pkd', 'wb'))

In [98]:
# Unpickle name-matching dictionary
import dill
Price_Patent_Data = dill.load(open('data/Price_Patent_Data.pkd', 'rb'))

In [99]:
# We'll drop these for now, but it may be useful to come back and get them for further analysis
Price_Patent_Data.drop([
                        'df;route', 
                        'appl_type_y',
                        'patent_no',
                        'appl_no',
                        'product_no',
                        'drug_substance_flag', 
                        'drug_product_flag', 
                        'patent_use_code',
                        'appl_type',
                        'delist_flag', 
                        'exclusivity_code'
                         ], axis = 1, inplace = True)

In [100]:
Price_Patent_Data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6746732 entries, 0 to 6746731
Data columns (total 28 columns):
key_0                                        6746732 non-null object
classification_for_rate_setting              6746732 non-null object
corresponding_generic_drug_effective_date    6746732 non-null object
corresponding_generic_drug_nadac_per_unit    6746732 non-null float64
effective_date                               6746732 non-null datetime64[ns]
explanation_code                             6746732 non-null object
nadac_per_unit                               6746732 non-null float64
ndc                                          6746732 non-null float64
ndc_description                              6746732 non-null object
otc                                          6746732 non-null object
pharmacy_type_indicator                      6746732 non-null object
pricing_unit                                 6746732 non-null object
ingredient                                   125

In [101]:
# Sort columns by ndc and effective date (for proper processing of ffill and bfill methods)
Price_Patent_Data.sort_values(['ndc', 'effective_date'], ascending = True, inplace = True)
Price_Patent_Data.head()

,key_0,classification_for_rate_setting,corresponding_generic_drug_effective_date,corresponding_generic_drug_nadac_per_unit,effective_date,explanation_code,nadac_per_unit,ndc,ndc_description,otc,...,approval_date,rld,rs,type,applicant_full_name,patent_expire_date_text,submission_date,exclusivity_date,dosage_form,route
0,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-21,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-22,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-23,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-24,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CALCITRIOL 0.25MCG ORAL,G,01/21/2015 12:00:00 AM,191.425,2013-11-25,1,0.55611,54000713.0,CALCITRIOL 0.25MCG CAPSULE,N,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### __Forward Fill New Rows with Existing Data__
Remember that we created a bunch of new rows/observations to accommodate the merging of price and patent information.  We can fill in the existing spaces with data from previous cells so that the transition from a historical cell to a current or future cell represents an appropriate change to a particular drug.

In [102]:
# Foward fill data the following columns -- MUST BE DONE BY NDC (forward fill will put current row's data into next row's null values)
# This is ok because none of the following data is expected to change (with the exception of patent_expire_date_text, which should be updated already in the table)
cols = ['ingredient', #f/bfill within drug
        'trade_name',  #f/bfill within drug
        'applicant', #f/bfill within drug
        'strength', #f/bfill within drug (because different ndc #s represent different dosages)
        'appl_type_x', #f/bfill within drug
        'te_code',  #f/bfill within drug (shouldn't change)
        'approval_date',#FFILL ONLY
        'rld',  #f/bfill within drug
        'rs', #f/bfill within drug
        'type', #f/bfill within drug
        'applicant_full_name', #f/bfill within drug (duplicate to 'applicant')
        'patent_expire_date_text',  #FFILL ONLY
        'submission_date',  #FFILL ONLY
        #'appl_type', #f/bfill within drug
        'exclusivity_date',  #FFILL ONLY
        'dosage_form', #f/bfill within drug
        'route'] #f/bfill within drug
# Price_Patent_Data.update(Price_Patent_Data.groupby('ndc')[cols].ffill())
Price_Patent_Data.update(Price_Patent_Data.groupby('ndc')[cols].fillna(method = 'ffill'))

In [103]:
Price_Patent_Data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6746732 entries, 0 to 6746731
Data columns (total 28 columns):
key_0                                        6746732 non-null object
classification_for_rate_setting              6746732 non-null object
corresponding_generic_drug_effective_date    6746732 non-null object
corresponding_generic_drug_nadac_per_unit    6746732 non-null float64
effective_date                               6746732 non-null datetime64[ns]
explanation_code                             6746732 non-null object
nadac_per_unit                               6746732 non-null float64
ndc                                          6746732 non-null float64
ndc_description                              6746732 non-null object
otc                                          6746732 non-null object
pharmacy_type_indicator                      6746732 non-null object
pricing_unit                                 6746732 non-null object
ingredient                                   387

In [55]:
#--------------------------------# 
# NOT BACKFILLING OUT OF CAUTION #  ==> # I believe that this may be ok, but because I'm not 100% sure, I'm leaving this out of the minimum viable product
#--------------------------------#

# cols = ['ingredient', #f/bfill within drug
#         'trade_name',  #f/bfill within drug
#         'applicant', #f/bfill within drug
#         'strength', #f/bfill within drug (because different ndc #s represent different dosages)
#         'appl_type_x', #f/bfill within drug
#         'te_code',  #f/bfill within drug (shouldn't change)
#         'rld',  #f/bfill within drug
#         'rs', #f/bfill within drug
#         'type', #f/bfill within drug
#         'applicant_full_name', #f/bfill within drug (duplicate to 'applicant')
#         'appl_type', #f/bfill within drug
#         'dosage_form', #f/bfill within drug
#         'route'] #f/bfill within drug
# # Price_Patent_Data.update(Price_Patent_Data.groupby('ndc')[cols].bfill()) #Can I do this without compromising data integrity?
# Price_Patent_Data.update(Price_Patent_Data.groupby('ndc')[cols].fillna(method = 'bfill'))

### __Delete Old Patent Data__
Price data only goes back to 2013-02-21; therefore any patent data older than that will not aid in the prediction of prices.  It is being removed in the next few lines.

It's been added to the 'To_do' notebook to come back and minimize data lost from this process.

In [ ]:
# Get minimum effective_date (earliest date for which I have price data)
min_effective_date = all_effective_dates.effective_date.min()
# Drop all observations with approval dates prior to 2013-02-21 (the earliest date of price information)
Cleaned_Patent_Data = Cleaned_Patent_Data[Cleaned_Patent_Data.approval_date >= min_effective_date]

In [ ]:
Price_Patent_Data.info(verbose = True, null_counts = True)

In [104]:
Price_Patent_Data = Price_Patent_Data.drop_duplicates(keep = 'first')
Price_Patent_Data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6746728 entries, 0 to 6746731
Data columns (total 28 columns):
key_0                                        6746728 non-null object
classification_for_rate_setting              6746728 non-null object
corresponding_generic_drug_effective_date    6746728 non-null object
corresponding_generic_drug_nadac_per_unit    6746728 non-null float64
effective_date                               6746728 non-null datetime64[ns]
explanation_code                             6746728 non-null object
nadac_per_unit                               6746728 non-null float64
ndc                                          6746728 non-null float64
ndc_description                              6746728 non-null object
otc                                          6746728 non-null object
pharmacy_type_indicator                      6746728 non-null object
pricing_unit                                 6746728 non-null object
ingredient                                   387

## __Further Cleaning__

#### __Delete all unneeded columns__

In [105]:
# Keep only the following:
cols = ['nadac_per_unit',            # Put pricing column in front for easy slicing (dependent variable in regression)
        'key_0',                     # Drug names (names matching patent dataset)
        'ndc',                       # Uniquely identifies drugs (including differing dosages & routes)
        'effective_date',
        'classification_for_rate_setting',
        'corresponding_generic_drug_effective_date',
        'corresponding_generic_drug_nadac_per_unit',
        'otc',
        'approval_date',             # These two date columns are for createing a 'days_before_patent_expires' column
        'patent_expire_date_text',   #
        'pricing_unit',
        'ingredient',
        'applicant',
        'submission_date',           # submission-application dates == 'days to process app' column
        'te_code',
        'type',
        #'exclusivity_code',
        'dosage_form',
        'route']
# Delete all columns not in the 'cols' list
Price_Patent_Data = Price_Patent_Data[cols]

In [106]:
Price_Patent_Data.rename({'key_0':'drug_names'}, axis = 1, inplace = True)

In [107]:
#Convert all to datetimes
Price_Patent_Data['effective_date'] = pd.to_datetime(Price_Patent_Data['effective_date']) #, format='%Y/%m/%d')
Price_Patent_Data['corresponding_generic_drug_effective_date'] = pd.to_datetime(Price_Patent_Data['corresponding_generic_drug_effective_date']) #, format='%Y/%m/%d')
Price_Patent_Data['approval_date'] = pd.to_datetime(Price_Patent_Data['approval_date']) #, format='%Y/%m/%d')
Price_Patent_Data['patent_expire_date_text'] = pd.to_datetime(Price_Patent_Data['patent_expire_date_text']) #, format='%Y/%m/%d')
Price_Patent_Data['submission_date'] = pd.to_datetime(Price_Patent_Data['submission_date']) #, format='%Y/%m/%d')
# Price_Patent_Data['exclusivity_date'] = pd.to_datetime(Price_Patent_Data['exclusivity_date']) #, format='%Y/%m/%d')

In [108]:
Price_Patent_Data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6746728 entries, 0 to 6746731
Data columns (total 18 columns):
nadac_per_unit                               6746728 non-null float64
drug_names                                   6746728 non-null object
ndc                                          6746728 non-null float64
effective_date                               6746728 non-null datetime64[ns]
classification_for_rate_setting              6746728 non-null object
corresponding_generic_drug_effective_date    6746728 non-null datetime64[ns]
corresponding_generic_drug_nadac_per_unit    6746728 non-null float64
otc                                          6746728 non-null object
approval_date                                3876079 non-null datetime64[ns]
patent_expire_date_text                      2582 non-null datetime64[ns]
pricing_unit                                 6746728 non-null object
ingredient                                   3876079 non-null object
applicant                  

## **Note:** The following overwrites current files
Be sure you're done with this notebook before continuing

In [109]:
# Pickle data
dill.dump(Price_Patent_Data, open('data/merged.pkd', 'wb'))

In [ ]:
# Export to csv so it can later be imported in a chunked fashion
Price_Patent_Data.to_csv('data/Price_Patent_Data.csv', index = False)

### __Get top 100 drugs by frequency of ndc__

In [110]:
# Another attempt at top 100 ndc numbers by frequency
top_100_drugs = Price_Patent_Data[Price_Patent_Data['ndc'].isin(Price_Patent_Data['ndc'].value_counts().index[:100])]
len(top_100_drugs.ndc.unique())

100

In [111]:
top_100_drugs.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214541 entries, 68472 to 6411864
Data columns (total 18 columns):
nadac_per_unit                               214541 non-null float64
drug_names                                   214541 non-null object
ndc                                          214541 non-null float64
effective_date                               214541 non-null datetime64[ns]
classification_for_rate_setting              214541 non-null object
corresponding_generic_drug_effective_date    214541 non-null datetime64[ns]
corresponding_generic_drug_nadac_per_unit    214541 non-null float64
otc                                          214541 non-null object
approval_date                                133310 non-null datetime64[ns]
patent_expire_date_text                      2211 non-null datetime64[ns]
pricing_unit                                 214541 non-null object
ingredient                                   133310 non-null object
applicant                          

In [112]:
# Pickle data
dill.dump(top_100_drugs, open('data/top_100_drugs.pkd', 'wb'))

## __Plotting Distribution of Data__

In [ ]:
#Checking distribution of prices before and after the addition of effective_dates (it appears they - and maybe all columns - were forward-filled as anticipated)
import matplotlib.pyplot as plt
# fig = plt.subplots(2, 1)
fig, (ax1, ax2) = plt.subplots(2, sharex = True)
ax1.hist(Cleaned_Price_Data['nadac_per_unit'])
ax1.set_title('Pre-dates-added')

ax2.hist(all_effective_dates['nadac_per_unit'])
ax2.set_title('Post-dates-added')
plt.show()

In [ ]:
def date_distribution(dataframe, column_as_string):
    import datetime as dt
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    
    dataframe[column_as_string] = pd.to_datetime(dataframe[column_as_string], errors = 'ignore').dt.date
    counts = dataframe[column_as_string].value_counts()

    fig, ax = plt.subplots(1,1, figsize = (10, 7))

    ax.hist(counts.index)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y'))
    ax.xdate = True
    plt.title('Distribution of {} over time'.format(column_as_string))
    plt.xticks(rotation = 30)
    return plt.show()

In [ ]:
date_distribution(Cleaned_Patent_Data, 'patent_expire_date_text')

In [ ]:
date_distribution(Cleaned_Price_Data, 'effective_date')

In [ ]:
date_distribution(Cleaned_Price_Data, 'as_of_date')

In [ ]:
date_distribution(Cleaned_Patent_Data, 'approval_date')